# EPL Standings ETL Pipeline

This notebook demonstrates a complete ETL (Extract, Transform, Load) pipeline that:
- Extracts Premier League standings data from the API-Football REST API
- Transforms the JSON response into a structured pandas DataFrame
- Loads the clean data into a PostgreSQL database

**Season**: 2023/24  
**Data Source**: [API-Football](https://www.api-football.com/) via API Sports

In [24]:
# Import required libraries
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, text
from sqlalchemy.exc import ProgrammingError
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import UniqueConstraint
from dotenv import load_dotenv

import os
import json
import requests
import pandas as pd

In [25]:
# Load environment variables from .env file
load_dotenv()

True

In [26]:
# API Configuration
API_KEY = os.getenv("API_KEY")      # Your API-Football key from RapidAPI
API_HOST = os.getenv("API_HOST")    # API base URL
SEASON = 2023                        # League season (2023 = 2023/24 season)
LEAGUE = 39                          # Premier League ID (39 = English Premier League)

In [27]:
# Setup API request components
url = f"{API_HOST}/standings"

# Authentication header
headers = {"x-apisports-key": API_KEY}

# Query parameters
params = {
    "league": str(LEAGUE),  # Premier League
    "season": SEASON        # 2023/24 season
}

## 1. Extract: Fetch data from API

In [28]:
# Send GET request to API
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()  # Raise exception if request failed

print(f"API request successful (Status: {response.status_code})")

API request successful (Status: 200)


In [29]:
# Parse JSON response
data = response.json()

# Preview the response structure
print(f"API returned {len(data['response'])} league(s)")
data

API returned 1 league(s)


{'get': 'standings',
 'parameters': {'league': '39', 'season': '2023'},
 'errors': [],
 'results': 1,
 'paging': {'current': 1, 'total': 1},
 'response': [{'league': {'id': 39,
    'name': 'Premier League',
    'country': 'England',
    'logo': 'https://media.api-sports.io/football/leagues/39.png',
    'flag': 'https://media.api-sports.io/flags/gb-eng.svg',
    'season': 2023,
    'standings': [[{'rank': 1,
       'team': {'id': 50,
        'name': 'Manchester City',
        'logo': 'https://media.api-sports.io/football/teams/50.png'},
       'points': 91,
       'goalsDiff': 62,
       'group': 'Premier League',
       'form': 'WWWWW',
       'status': 'same',
       'description': 'Promotion - Champions League (Group Stage: )',
       'all': {'played': 38,
        'win': 28,
        'draw': 7,
        'lose': 3,
        'goals': {'for': 96, 'against': 34}},
       'home': {'played': 19,
        'win': 14,
        'draw': 5,
        'lose': 0,
        'goals': {'for': 51, 'against': 1

In [30]:
# Extract standings data from nested response
# API structure: response -> league -> standings -> [0] (main table)
standings = data["response"][0]["league"]["standings"][0]

print(f"Extracted standings for {len(standings)} teams")
print(f"\nSample team data structure:")
print(json.dumps(standings[0], indent=2))

Extracted standings for 20 teams

Sample team data structure:
{
  "rank": 1,
  "team": {
    "id": 50,
    "name": "Manchester City",
    "logo": "https://media.api-sports.io/football/teams/50.png"
  },
  "points": 91,
  "goalsDiff": 62,
  "group": "Premier League",
  "form": "WWWWW",
  "status": "same",
  "description": "Promotion - Champions League (Group Stage: )",
  "all": {
    "played": 38,
    "win": 28,
    "draw": 7,
    "lose": 3,
    "goals": {
      "for": 96,
      "against": 34
    }
  },
  "home": {
    "played": 19,
    "win": 14,
    "draw": 5,
    "lose": 0,
    "goals": {
      "for": 51,
      "against": 16
    }
  },
  "away": {
    "played": 19,
    "win": 14,
    "draw": 2,
    "lose": 3,
    "goals": {
      "for": 45,
      "against": 18
    }
  },
  "update": "2024-05-28T00:00:00+00:00"
}


## 2. Transform: Parse response into dataframe

In [31]:
# Transform nested JSON into flat table structure
rows = []
columns = [
    'season', 'position', 'team_id', 'team_name', 'team_logo', 
    'played', 'won', 'draw', 'lose', 'goals_for', 'goals_against', 
    'goal_difference', 'points', 'form', 'description'
]

# Extract relevant fields from each team's data
for club in standings:
    try:
        season = SEASON
        position = club['rank']
        team_id = club['team']['id']
        team_name = club['team']['name']
        team_logo = club['team']['logo']
        played = club['all']['played']
        won = club['all']['win']
        draw = club['all']['draw']
        lose = club['all']['lose']
        goals_for = club['all']['goals']['for']
        goals_against = club['all']['goals']['against']
        goal_difference = club['goalsDiff']
        points = club['points']
        form = club['form']  # Last 5 matches (e.g., "WWDLW")
        description = club['description']  # e.g., "Promotion - Champions League"
        
        # Add row as tuple
        active_rows = (
            season, position, team_id, team_name, team_logo, 
            played, won, draw, lose, goals_for, goals_against, 
            goal_difference, points, form, description
        )
        rows.append(active_rows)
        
    except KeyError as e:
        print(f"Missing key in data structure: {e}")
        print(f"Problem record: {club}")

print(f"Transformed {len(rows)} team records")
rows[:3]  # Preview first 3 rows

Transformed 20 team records


[(2023,
  1,
  50,
  'Manchester City',
  'https://media.api-sports.io/football/teams/50.png',
  38,
  28,
  7,
  3,
  96,
  34,
  62,
  91,
  'WWWWW',
  'Promotion - Champions League (Group Stage: )'),
 (2023,
  2,
  42,
  'Arsenal',
  'https://media.api-sports.io/football/teams/42.png',
  38,
  28,
  5,
  5,
  91,
  29,
  62,
  89,
  'WWWWW',
  'Promotion - Champions League (Group Stage: )'),
 (2023,
  3,
  40,
  'Liverpool',
  'https://media.api-sports.io/football/teams/40.png',
  38,
  24,
  10,
  4,
  86,
  41,
  45,
  82,
  'WDWDL',
  'Promotion - Champions League (Group Stage: )')]

In [32]:
# Create pandas DataFrame
df = pd.DataFrame(rows, columns=columns)

print(f"DataFrame shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head()

DataFrame shape: (20, 15)
Columns: ['season', 'position', 'team_id', 'team_name', 'team_logo', 'played', 'won', 'draw', 'lose', 'goals_for', 'goals_against', 'goal_difference', 'points', 'form', 'description']


,season,position,team_id,team_name,team_logo,played,won,draw,lose,goals_for,goals_against,goal_difference,points,form,description
0,2023,1,50,Manchester City,https://media.api-sports.io/football/teams/50.png,38,28,7,3,96,34,62,91,WWWWW,Promotion - Champions League (Group Stage: )
1,2023,2,42,Arsenal,https://media.api-sports.io/football/teams/42.png,38,28,5,5,91,29,62,89,WWWWW,Promotion - Champions League (Group Stage: )
2,2023,3,40,Liverpool,https://media.api-sports.io/football/teams/40.png,38,24,10,4,86,41,45,82,WDWDL,Promotion - Champions League (Group Stage: )
3,2023,4,66,Aston Villa,https://media.api-sports.io/football/teams/66.png,38,20,8,10,76,61,15,68,LDLDW,Promotion - Champions League (Group Stage: )
4,2023,5,47,Tottenham,https://media.api-sports.io/football/teams/47.png,38,20,6,12,74,61,13,66,WLWLL,Promotion - Europa League (Group Stage: )


In [33]:
# Data quality checks and cleaning
print("Checking for missing values...")
print(df.isnull().sum())

# Fill null descriptions (teams mid-table without qualification status)
df['description'] = df['description'].fillna('EPL: Next Season')

print(f"\nData cleaning complete")
df.head(10)

Checking for missing values...
season              0
position            0
team_id             0
team_name           0
team_logo           0
played              0
won                 0
draw                0
lose                0
goals_for           0
goals_against       0
goal_difference     0
points              0
form                0
description        10
dtype: int64

Data cleaning complete


,season,position,team_id,team_name,team_logo,played,won,draw,lose,goals_for,goals_against,goal_difference,points,form,description
0,2023,1,50,Manchester City,https://media.api-sports.io/football/teams/50.png,38,28,7,3,96,34,62,91,WWWWW,Promotion - Champions League (Group Stage: )
1,2023,2,42,Arsenal,https://media.api-sports.io/football/teams/42.png,38,28,5,5,91,29,62,89,WWWWW,Promotion - Champions League (Group Stage: )
2,2023,3,40,Liverpool,https://media.api-sports.io/football/teams/40.png,38,24,10,4,86,41,45,82,WDWDL,Promotion - Champions League (Group Stage: )
3,2023,4,66,Aston Villa,https://media.api-sports.io/football/teams/66.png,38,20,8,10,76,61,15,68,LDLDW,Promotion - Champions League (Group Stage: )
4,2023,5,47,Tottenham,https://media.api-sports.io/football/teams/47.png,38,20,6,12,74,61,13,66,WLWLL,Promotion - Europa League (Group Stage: )
5,2023,6,49,Chelsea,https://media.api-sports.io/football/teams/49.png,38,18,9,11,77,63,14,63,WWWWW,Promotion - Europa Conference League (Qualific...
6,2023,7,34,Newcastle,https://media.api-sports.io/football/teams/34.png,38,18,6,14,85,62,23,60,WLDWW,EPL: Next Season
7,2023,8,33,Manchester United,https://media.api-sports.io/football/teams/33.png,38,18,6,14,57,58,-1,60,WWLLD,Promotion - Europa League (Group Stage: )
8,2023,9,48,West Ham,https://media.api-sports.io/football/teams/48.png,38,14,10,14,60,74,-14,52,LWLDL,EPL: Next Season
9,2023,10,52,Crystal Palace,https://media.api-sports.io/football/teams/52.png,38,13,10,15,57,58,-1,49,WWWDW,EPL: Next Season


## 3. Load: Upload data into PostgreSQL

In [34]:
# PostgreSQL database configuration
DB_CONFIG = {
    "host": os.getenv("HOST"),
    "port": os.getenv("PORT"),
    "database": os.getenv("DB"),
    "user": os.getenv("USER"),
    "password": os.getenv("PASSWORD"),
}

print("Database configuration loaded")
print(f"Target: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

Database configuration loaded
Target: localhost:5432/epl_standings


In [35]:
# === STEP 1: Create database if it doesn't exist ===
admin_engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}",
    isolation_level='AUTOCOMMIT'  # Required for CREATE DATABASE command
)

with admin_engine.connect() as conn:
    try:
        conn.execute(text(f"CREATE DATABASE {DB_CONFIG['database']}"))
        print(f"Database '{DB_CONFIG['database']}' created successfully")
    except ProgrammingError as e:
        if "already exists" in str(e):
            print(f"Database '{DB_CONFIG['database']}' already exists")
        else:
            raise e

# === STEP 2: Connect to the target database ===
engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}",
    echo=False,  # Set to True to see SQL queries
    pool_timeout=10
)

print("Connected to database")

# === STEP 3: Define table schema ===
metadata = MetaData()

standings_table = Table(
    "standings",
    metadata,
    # Primary keys (composite: season + team_id)
    Column("season", Integer, primary_key=True, nullable=False),
    Column("team_id", Integer, primary_key=True, nullable=False),
    
    # Team information
    Column("position", Integer, nullable=False),
    Column("team_name", String(100), nullable=False),
    Column("team_logo", String(100), nullable=False),
    
    # Match statistics
    Column("played", Integer, nullable=False),
    Column("won", Integer, nullable=False),
    Column("draw", Integer, nullable=False),
    Column("lose", Integer, nullable=False),
    
    # Goals
    Column("goals_for", Integer, nullable=False),
    Column("goals_against", Integer, nullable=False),
    Column("goal_difference", Integer, nullable=False),
    
    # Standings info
    Column("points", Integer, nullable=False),
    Column("form", String(5), nullable=False),  # Last 5 matches
    Column("description", String(100), default="EPL: Next Season", nullable=False),
    
    # Ensure unique position per season
    UniqueConstraint("season", "position", name="uix_season_position"),
)

# === STEP 4: Create table ===
metadata.create_all(engine)
print("Table 'standings' created/verified successfully")

Database 'epl_standings' already exists
Connected to database
Table 'standings' created/verified successfully


In [36]:
# === UPSERT data into database ===
# Convert DataFrame to list of dictionaries
df_rows = df.to_dict(orient="records")

# Create INSERT statement
stmt = insert(standings_table)

# Create UPSERT statement (INSERT ... ON CONFLICT DO UPDATE)
# This updates existing records or inserts new ones
upsert_stmt = stmt.on_conflict_do_update(
    index_elements=['season', 'team_id'],  # Conflict on these columns
    set_={c.name: c for c in stmt.excluded if c.name not in ['season', 'team_id']}
)

# Execute upsert
with engine.begin() as conn:
    try:    
        result = conn.execute(upsert_stmt, df_rows)
        print(f"Successfully upserted {len(df_rows)} records")
    except Exception as e:
        print(f"Error upserting data: {e}")
        raise

Successfully upserted 20 records


In [37]:
# === Verify data was loaded correctly ===
with engine.connect() as conn:
    # Query all records from standings table
    result = conn.execute(standings_table.select().order_by(standings_table.c.position))
    
    print(f"Current standings in database:\n")
    print(f"{'Pos':<5} {'Team':<25} {'P':<4} {'W':<4} {'D':<4} {'L':<4} {'GD':<5} {'Pts':<5}")
    print("-" * 60)
    
    for row in result:
        print(f"{row.position:<5} {row.team_name:<25} {row.played:<4} {row.won:<4} "
              f"{row.draw:<4} {row.lose:<4} {row.goal_difference:<+5} {row.points:<5}")
    
print(f"\nETL Pipeline Complete!")

Current standings in database:

Pos   Team                      P    W    D    L    GD    Pts  
------------------------------------------------------------
1     Manchester City           38   28   7    3    +62   91   
2     Arsenal                   38   28   5    5    +62   89   
3     Liverpool                 38   24   10   4    +45   82   
4     Aston Villa               38   20   8    10   +15   68   
5     Tottenham                 38   20   6    12   +13   66   
6     Chelsea                   38   18   9    11   +14   63   
7     Newcastle                 38   18   6    14   +23   60   
8     Manchester United         38   18   6    14   -1    60   
9     West Ham                  38   14   10   14   -14   52   
10    Crystal Palace            38   13   10   15   -1    49   
11    Brighton                  38   12   12   14   -7    48   
12    Bournemouth               38   13   9    16   -13   48   
13    Fulham                    38   13   8    17   -6    47   
14    Wolve